In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
%matplotlib inline


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

In [2]:
def answer_one():
    
    g = nx.read_adjlist('Employee_Movie_Choices.txt', delimiter='\t')
#     plt.figure(figsize=(10, 10))
#     nx.draw_networkx(g)
#     print('nodes: {}\nedges: {}'.format(len(g.nodes()), len(g.edges()) ) )
#     print(nx.bipartite.is_bipartite(g))

    return g

In [3]:
def answer_two():
    g = answer_one()
    for i in employees:
        g.add_node(i, type="employee")
    for i in movies:
        g.add_node(i, type="movie")
#     print(g.nodes(data=True))
    return g

In [4]:
def answer_three():
    p = bipartite.weighted_projected_graph(answer_two(), employees)
#     print(p.edges(data=True))
    return p

In [5]:
def answer_four():

    g = answer_three()
    
    common = pd.DataFrame(data=g.edges(data=True), columns=['emp1', 'emp2', 'numOfMovies'])
    common['numOfMovies'] = common['numOfMovies'].apply(lambda x: x['weight'])
    
    h = nx.read_edgelist(path='Employee_Relationships.txt', data=[('relScore', int)], delimiter='\t')
    rel = pd.DataFrame(data=h.edges(data=True), columns=['emp1', 'emp2', 'score'])
    rel['score'] = rel['score'].apply(lambda x: x['relScore'])
    
    merged = pd.merge(left=rel, right=common, how='left', left_on=['emp1','emp2'], right_on=['emp1','emp2'])
    merged.fillna(value=0, inplace=True)
    
    corr = merged[['score','numOfMovies']].corr()
    movieRelCorr = corr['score']['numOfMovies']
    
    return float(movieRelCorr)

EOF